In [114]:
#This file aims to produce a real case scenario of an incident report
#The report will be wrote into a json file

In [115]:
import json
import random
import numpy as np
from Incident import Incident
from docplex.mp.model import Model
from qiskit import BasicAer
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.utils.algorithm_globals import algorithm_globals
from qiskit_optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.problems.variable import VarType
from qiskit_optimization.converters.quadratic_program_to_qubo import QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp
import dwave

In [140]:
estimated_times = [6,24,8,40,24,8,24,2,6,72,16,8]

In [116]:
#First of all, I've to instanciate dictionaries of controls and incidents
controls = {
    #control_key : [control_name, estimated_time]
    1 : ["Information Backup", 6],
    2 : ["Capacity Management", 6],
    3 : ["Off-Site Equipment Security", 24],
    4 : ["Physical Entry Controls", 8],
    5 : ["Restriction Of Access To Information", 40],
    6 : ["Management Of Secret Information Authentication", 40],
    7 : ["Equipment Maintenance", 24],
    8 : ["Capacity Management", 8],
    9 : ["Clock Synchronization", 2],
    11 : ["Supply Facilities", 72],
    12 : ["Network Controls", 8]
}

In [117]:
threats = {
    #threat_key : [threat_name, control_key1, control_key2, ...]
    1 : ["Denial Of Service", 1, 2],
    2 : ["Theft", 3],
    3 : ["Social Engineering", 4, 6],
    4 : ["Monitoring Errors", 5, 6],
    5 : ["Failure Of Physical Or Logical Origin", 7],
    6 : ["System Crashes Due To Resource Exhaustion", 8],
    7 : ["Abuse Of Access Privileges", 5],
    8 : ["Configuration Errors", 9],
    9 : ["Maintenance Errors", 1],
    10 : ["Inadequate Temperature Or Humidity Conditions", 11],
    11 : ["Failure Of Communication Services", 12]
}

In [118]:
penality = 73

In [119]:
#Now I've to make the dataset (of 14 incidents)
incidents = []
for _ in range(14):
    threat_key = random.randint(1,11)
    threat_list = threats.get(threat_key)
    #if the list contains just the threat name, we have to choose another threat
    while(len(threat_list) < 2):
        threat_key = random.randint(1,11)
        threat_list = threats.get(threat_key)
    threat_name = threat_list[0]
    control_key = threat_list.pop(random.randint(1,len(threat_list)-1))
    control_list = controls[control_key]
    control_name = control_list[0]
    estimated_time = control_list[1]
    incident = Incident(threat_key, threat_name, control_key,control_name, estimated_time)
    incidents.append(incident)

for i in range(len(incidents)):
    print(incidents[i])

threat_id: 1, threat_name: Denial Of Service, control_id: 1, control_name: Information Backup, estimated_time: 6
threat_id: 7, threat_name: Abuse Of Access Privileges, control_id: 5, control_name: Restriction Of Access To Information, estimated_time: 40
threat_id: 3, threat_name: Social Engineering, control_id: 4, control_name: Physical Entry Controls, estimated_time: 8
threat_id: 2, threat_name: Theft, control_id: 3, control_name: Off-Site Equipment Security, estimated_time: 24
threat_id: 10, threat_name: Inadequate Temperature Or Humidity Conditions, control_id: 11, control_name: Supply Facilities, estimated_time: 72
threat_id: 11, threat_name: Failure Of Communication Services, control_id: 12, control_name: Network Controls, estimated_time: 8
threat_id: 1, threat_name: Denial Of Service, control_id: 2, control_name: Capacity Management, estimated_time: 6
threat_id: 6, threat_name: System Crashes Due To Resource Exhaustion, control_id: 8, control_name: Capacity Management, estimated_

In [142]:
#mi serve una lista di incidenti per ogni controllo
actual_controls = {}
for incident in incidents:
    control_id = incident.get_control_id()
    if actual_controls.keys().__contains__(control_id):
        threat_list = actual_controls.get(control_id)
        threat_list.append(incident.get_threat_id())
        threat_list.sort()
    else:
        actual_controls[control_id] = [incident.get_threat_id()]

print("Actual affected controls:" + str(actual_controls))

Actual affected controls:{1: [1, 9], 5: [4, 7], 4: [3], 3: [2], 11: [10], 12: [11], 2: [1], 8: [6], 6: [3, 4], 9: [8], 7: [5]}


In [121]:
def create_QUBO_problem(linear_terms,quadratic_terms):
    model = Model()

    #i nomi delle var lineare saranno x1,...,xn. Rappresentano il tempo stimato per ogni incidente in esame
    #x partirà però sempre da 0 (x1 = x[0])
    x = [model.binary_var("x%s" % (i)) for i in range(1,len(quadratic_terms)+1)]

    #per prima cosa aggiungiamo i bias (diagonale dei valori lineari della matrice)
    #bias = estimated_time - penality
    #iniziamo con l'aggiunta degli estimated_time
    objective = model.sum([linear_terms[i] * x[i] for i in range(len(linear_terms))])

    #adesso, per ogni pericolo relativo ai controlli colpiti, sottraiamo la penality
    for threat_list in actual_controls.values():
        for threat_id in threat_list:
            objective2 = model.sum(-penality*x[threat_id-1])
            objective += objective2

    #aggiungiamo i pesi di accoppiamento (triangolo superiore con i valori quadratici della matrice)
    objective += model.sum([((2*quadratic_terms[i, j]) * x[i] * x[j]) for i in range(len(linear_terms)) for j in range(len(quadratic_terms))])

    #vogliamo minimizzare l'hamiltoniana
    model.minimize(objective)

    #convertiamo il modello in un modello QUBO
    qubo = from_docplex_mp(model)

    cost = model.sum(x)
    model.add_constraint(cost == 3)

    return qubo

In [191]:
#generiamo i termini lineari
linear_terms = np.array(estimated_times)

#generiamo i termini quadratici
quadratic_dictionary = {
    1 : [0,0,0,0,0,0,0,0,0,0,0,0],
    2 : [0,0,0,0,0,0,0,0,0,0,0,0],
    3 : [0,0,0,0,0,0,0,0,0,0,0,0],
    4 : [0,0,0,0,0,0,0,0,0,0,0,0],
    5 : [0,0,0,0,0,0,0,0,0,0,0,0],
    6 : [0,0,0,0,0,0,0,0,0,0,0,0],
    7 : [0,0,0,0,0,0,0,0,0,0,0,0],
    8 : [0,0,0,0,0,0,0,0,0,0,0,0],
    9 : [0,0,0,0,0,0,0,0,0,0,0,0],
    10 : [0,0,0,0,0,0,0,0,0,0,0,0],
    11 : [0,0,0,0,0,0,0,0,0,0,0,0],
    12 : [0,0,0,0,0,0,0,0,0,0,0,0]
}
for threat_list in actual_controls.values():
    for i in threat_list:
        for j in threat_list:
            if(j>i):
                quadratic_dictionary[i][j-1]+=penality

quadratic_terms = []
for vector in quadratic_dictionary.values():
    quadratic_terms.append(vector)
quadratic_terms = np.array(quadratic_terms)

[ 6 24  8 40 24  8 24  2  6 72 16  8]
[[ 0  0  0  0  0  0  0  0 73  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 73  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 73  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0]]


In [192]:
qubo = create_QUBO_problem(linear_terms,quadratic_terms)
qubo

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model40

Minimize
 obj: - 140 x1 - 49 x2 - 138 x3 - 106 x4 - 49 x5 - 65 x6 - 49 x7 - 71 x8 - 67 x9
      - x10 - 57 x11 + 8 x12 + [ 292 x1*x9 + 292 x3*x4 + 292 x4*x7 ]/2
Subject To

Bounds
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1
 0 <= x5 <= 1
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1
 0 <= x12 <= 1

Binaries
 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11 x12
End